# Chuẩn bị dư liệu

In [1]:
import pandas as pd
import networkx as nx
from networkx.algorithms import approximation as approx
from networkx.algorithms.link_prediction import jaccard_coefficient, adamic_adar_index, preferential_attachment
import random


In [2]:
# Đọc file coauthor
df = pd.read_csv(r"C:\Users\thean\Documents\Social_Network\Social_Network_Projects\Dataset\Project_1_coauthor_edges.csv")

# Chuyển Weight về int
df["Weight"] = df["Weight"].astype(str).str.extract(r"(\d+)").astype(int)

df.head()

,Author1,Author2,Weight
0,Joseph Sambrook,Elisabeth Fritsch,1
1,Joseph Sambrook,Tom Maniatis,1
2,Elisabeth Fritsch,Tom Maniatis,1
3,Keith E. Muller,Jacob Cohen,1
4,Jacob Cohen,Peter A. Lachenbruch,1


In [3]:
# Tạo graph
G = nx.from_pandas_edgelist(df, "Author1", "Author2", edge_attr="Weight")

print(G.is_directed())
print("Số nodes:", G.number_of_nodes())
print("Số edges:", G.number_of_edges())
print("Có self-loop không:", nx.number_of_selfloops(G))
print("Có duplicate edges không:", len(df) - G.number_of_edges())

False
Số nodes: 9212
Số edges: 136198
Có self-loop không: 30
Có duplicate edges không: 69


Dataset được tính Nodes bằng Author_ID và lưu lại bằng tên nên trường hợp 2 tác giả có tên giống nhau cùng hợp tác viết một bài báo hay là một tác giả hợp tác với hai người khác nhau trùng tên là hoàn toàn có thể xảy ra, vì vậy dù kiểm tra cho thấy có self-loop và duplicate edges trong mạng hợp tác ta bỏ qua và tiến hành bước tiếp theo

In [4]:
# Kiểm tra mạng liên thông (connected components)
components = sorted(nx.connected_components(G), key=len, reverse=True)
print("Số thành phần liên thông:", len(components))
print("Kích thước thành phần lớn nhất:", len(components[0]))
# In thống kê top 10 component lớn nhất
print("Top 10 connected components:")
for i, comp in enumerate(components[:10]):
    n_nodes = len(comp)
    n_edges = len(G.subgraph(comp).edges())
    density = nx.density(G.subgraph(comp))
    print(f"Component {i+1:2d}: {n_nodes:5d} nodes, {n_edges:7d} edges, density={density:.4f}")

Số thành phần liên thông: 1145
Kích thước thành phần lớn nhất: 2616
Top 10 connected components:
Component  1:  2616 nodes,   76666 edges, density=0.0224
Component  2:   266 nodes,    7362 edges, density=0.2089
Component  3:   211 nodes,    8129 edges, density=0.3669
Component  4:   124 nodes,     503 edges, density=0.0660
Component  5:   117 nodes,    5103 edges, density=0.7520
Component  6:   105 nodes,    4959 edges, density=0.9082
Component  7:    99 nodes,    4851 edges, density=1.0000
Component  8:    98 nodes,    2909 edges, density=0.6120
Component  9:    88 nodes,    2954 edges, density=0.7717
Component 10:    84 nodes,    1084 edges, density=0.3110


## Nhận xét

- Số thành phần liên thông (connected components): 1145, Mạng tác giả bị phân tách mạnh -> có nhiều cụm tác giả tách biệt.

- Kích thước thành phần lớn nhất (giant component): 2616, thành phần lớn nhất chiếm khoảng 28% tổng số node (2611/9212)

Chỉ số trên cho thấy mạng phân mảnh cao, ngoài giant component còn có hơn 1100 nhóm nhỏ.

=> Điều này ảnh hưởng lớn đến khả năng dự đoán của Model, chỉ có thể dự đoán liên kết mới giữa các node cùng component

Nếu để cả graph, phần lớn node sẽ không có khả năng kết nối mới hợp lệ -> gây nhiễu hoặc bias nặng.

## Lọc component

In [5]:
selected_components = []

# Ngưỡng tùy chỉnh
# bỏ các nhóm quá nhỏ
MIN_NODES = 50
# bỏ nhóm quá dày đặc (density > 0.5)
MAX_DENSITY = 0.5

for i, comp in enumerate(components):
    subG = G.subgraph(comp)
    n_nodes = subG.number_of_nodes()
    n_edges = subG.number_of_edges()
    density = nx.density(subG)

    # Lọc theo tiêu chí
    if n_nodes >= MIN_NODES and density <= MAX_DENSITY:
        selected_components.append(subG)
        print(f"Component {i+1}: {n_nodes} nodes, {n_edges} edges, density={density:.4f}")
    else:
        print(f"Bỏ Component {i+1}: {n_nodes} nodes, {n_edges} edges, density={density:.4f}")

# Gộp các component đã chọn lại (thường chỉ có 1–3 cái)
G_lp = nx.compose_all(selected_components)

print("\nTổng kết sau khi lọc:")
print("Số component được giữ:", len(selected_components))
print("Tổng số node:", G_lp.number_of_nodes())
print("Tổng số edge:", G_lp.number_of_edges())
print("Độ đậm đặc:", nx.density(G_lp))


Component 1: 2616 nodes, 76666 edges, density=0.0224
Component 2: 266 nodes, 7362 edges, density=0.2089
Component 3: 211 nodes, 8129 edges, density=0.3669
Component 4: 124 nodes, 503 edges, density=0.0660
Bỏ Component 5: 117 nodes, 5103 edges, density=0.7520
Bỏ Component 6: 105 nodes, 4959 edges, density=0.9082
Bỏ Component 7: 99 nodes, 4851 edges, density=1.0000
Bỏ Component 8: 98 nodes, 2909 edges, density=0.6120
Bỏ Component 9: 88 nodes, 2954 edges, density=0.7717
Component 10: 84 nodes, 1084 edges, density=0.3110
Bỏ Component 11: 73 nodes, 2628 edges, density=1.0000
Bỏ Component 12: 58 nodes, 1653 edges, density=1.0000
Bỏ Component 13: 53 nodes, 741 edges, density=0.5377
Component 14: 52 nodes, 604 edges, density=0.4555
Bỏ Component 15: 49 nodes, 432 edges, density=0.3673
Bỏ Component 16: 39 nodes, 520 edges, density=0.7018
Bỏ Component 17: 36 nodes, 260 edges, density=0.4127
Bỏ Component 18: 34 nodes, 185 edges, density=0.3298
Bỏ Component 19: 32 nodes, 311 edges, density=0.6270
B

Sau khi lọc, đồ thị còn 6 thành phần liên thông với 3.353 nút và 94.348 cạnh, có độ đậm đặc 0.0168. Cấu trúc này cho thấy mạng vẫn đủ lớn và thưa, phù hợp cho bài toán dự đoán liên kết vì còn nhiều cặp nút tiềm năng chưa kết nối để mô hình học và dự đoán.

# Dự đoán liên kết

## Bước 1: Chia train-test cho Link Prediction
-   Trích một phần cạnh (khoảng 20%) làm test set.
-   Xây dựng train graph từ phần còn lại.

In [6]:
#Lấy tất cả các cạnh (các hợp tác hiện có giữa các tác giả) trong mạng G.
edges = list(G.edges())
#Đặt seed ngẫu nhiên để kết quả có thể lặp lại mỗi lần chạy.
random.seed(42)
#Xác định kích thước tập test bằng 20% tổng số cạnh.
test_size = int(0.2 * len(edges))
#Lấy ngẫu nhiên 20% cạnh làm tập test. Đây là những cạnh giả định bị ẩn để kiểm tra khả năng dự đoán sau này.
test_edges = random.sample(edges, test_size)

#Tạo bản sao của mạng để làm tập huấn luyện.
train_G = G.copy()
#Xóa các cạnh trong test set khỏi tập train, giả lập rằng chúng chưa tồn tại
train_G.remove_edges_from(test_edges)

#In ra số cạnh còn lại trong tập huấn luyện.
print("Số cạnh train:", train_G.number_of_edges())
#In ra số cạnh trong tập test.
print("Số cạnh test:", len(test_edges))


Số cạnh train: 108959
Số cạnh test: 27239


Đoạn code trên thực hiện việc chia dữ liệu mạng thành hai phần: **tập huấn luyện (80%)** và **tập kiểm tra (20%)** bằng cách chọn ngẫu nhiên 20% cạnh để làm **test set** và loại bỏ chúng khỏi mạng gốc. Việc đặt seed cố định giúp đảm bảo khả năng tái lập kết quả. Cách chia này mô phỏng quá trình “ẩn” các liên kết thật, tạo điều kiện để đánh giá năng lực dự đoán liên kết mới của mô hình.

## Bước 2: Tính các chỉ số dự đoán liên kết (Link Prediction Scores)

Bốn thuật toán Common Neighbors, Jaccard Coefficient, Adamic-Adar và Preferential Attachment được áp dụng để tính điểm tương đồng giữa các cặp tác giả chưa có liên kết trong mạng huấn luyện.
Các chỉ số này phản ánh mức độ gần gũi giữa hai nút dựa trên cấu trúc lân cận:

CN đo số lượng bạn chung,

JC chuẩn hóa theo tổng số bạn,

AA nhấn mạnh các bạn chung hiếm gặp,

PA giả định các nút có độ kết nối cao dễ liên kết hơn.

Các giá trị điểm cao cho thấy cặp tác giả có khả năng cao sẽ hình thành hợp tác trong tương lai.

In [7]:
# Common Neighbors
cn_scores = [(u, v, len(list(nx.common_neighbors(train_G, u, v))))
             for u, v in nx.non_edges(train_G)]
# In top 10 kết quả Common Neighbors
print("Common Neighbors (top 10):")
print(sorted(cn_scores, key=lambda x: x[2], reverse=True)[:10])

# Jaccard
jc_scores = list(jaccard_coefficient(train_G))
# In top 10 Jaccard Coefficient
print("\nJaccard Coefficient (top 10):")
print(sorted(jc_scores, key=lambda x: x[2], reverse=True)[:10])

# Adamic-Adar
aa_scores = list(adamic_adar_index(train_G))
# In top 10 Adamic-Adar Index
print("\nAdamic-Adar Index (top 10):")
print(sorted(aa_scores, key=lambda x: x[2], reverse=True)[:10])

# Preferential Attachment
pa_scores = list(preferential_attachment(train_G))
# In top 10 Preferential Attachment
print("\nPreferential Attachment (top 10):")
print(sorted(pa_scores, key=lambda x: x[2], reverse=True)[:10])



Common Neighbors (top 10):
[('J. Michael Cherry', 'Harold Drabkin', 126), ('Karen Christie', 'David P. Hill', 126), ('Penelope Garmiri', 'Rizwan Ishtiaq', 125), ('Marc Feuermann', 'Lucila Aimo', 124), ('Rex L. Chisholm', 'David P. Hill', 123), ('Stacia R. Engel', 'Karen Christie', 122), ('Chris Mungall', 'David P. Hill', 121), ('Robert S. Fulton', 'Asif Chinwalla', 118), ('M. Eileen Dolan', 'Rex L. Chisholm', 117), ('Pascale Gaudet', 'Lucila Aimo', 117)]

Jaccard Coefficient (top 10):
[('Abhishek Verma', 'John Wilkes', 1.0), ('G. T. Kokotailo', 'Stephen L. Lawton', 1.0), ('Virginia Smith', 'Ameet Talwalkar', 1.0), ('Geoffrey R. Norman', 'David L. Streiner', 1.0), ('Ruslan Mustafin', 'Manuel Mazzara', 1.0), ('Roger Strange', 'Kjell Grønhaug', 1.0), ('Thomas K. Dasaklis', 'Constantinos Patsakis', 1.0), ('Keith Woodward', 'John Paul Jones', 1.0), ('Elizabeth McInnes', 'Jonathan C. Craig', 1.0), ('Jung Kwon Lee', 'Hyunsoo Kim', 1.0)]

Adamic-Adar Index (top 10):
[('Karen Christie', 'David 

### Step 4: Lấy top dự đoán


In [8]:
#  Common Neighbors
cn_scores_sorted = sorted(cn_scores, key=lambda x: x[2], reverse=True)
top_pred = cn_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Common Neighbors):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Common Neighbors):
J. Michael Cherry - Harold Drabkin : 126
Karen Christie - David P. Hill : 126
Penelope Garmiri - Rizwan Ishtiaq : 125
Marc Feuermann - Lucila Aimo : 124
Rex L. Chisholm - David P. Hill : 123
Stacia R. Engel - Karen Christie : 122
Chris Mungall - David P. Hill : 121
Robert S. Fulton - Asif Chinwalla : 118
M. Eileen Dolan - Rex L. Chisholm : 117
Pascale Gaudet - Lucila Aimo : 117


In [9]:
#  Jaccard Coefficient
jc_scores_sorted = sorted(jc_scores, key=lambda x: x[2], reverse=True)
top_pred = jc_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Jaccard Coefficient):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Jaccard Coefficient):
Abhishek Verma - John Wilkes : 1.0
G. T. Kokotailo - Stephen L. Lawton : 1.0
Virginia Smith - Ameet Talwalkar : 1.0
Geoffrey R. Norman - David L. Streiner : 1.0
Ruslan Mustafin - Manuel Mazzara : 1.0
Roger Strange - Kjell Grønhaug : 1.0
Thomas K. Dasaklis - Constantinos Patsakis : 1.0
Keith Woodward - John Paul Jones : 1.0
Elizabeth McInnes - Jonathan C. Craig : 1.0
Jung Kwon Lee - Hyunsoo Kim : 1.0


In [10]:
#  Preferential Attachment
pa_scores_sorted = sorted(pa_scores, key=lambda x: x[2], reverse=True)
top_pred = pa_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Preferential Attachment):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Preferential Attachment):
Andrés Mauricio Caraballo‐Rodríguez - Michelle Giglio : 41193
Michelle Giglio - Ricardo Silva : 40572
Andrés Mauricio Caraballo‐Rodríguez - Pascale Gaudet : 35820
Pascale Gaudet - Ricardo Silva : 35280
Andrés Mauricio Caraballo‐Rodríguez - Curtis Huttenhower : 33233
Robert S. Fulton - Michelle Giglio : 32913
Michelle Giglio - Karen Christie : 32913
Curtis Huttenhower - Ricardo Silva : 32732
Michelle Giglio - Paul D. Thomas : 32706
Andrés Mauricio Caraballo‐Rodríguez - Sandra W. Clifton : 32636


In [11]:
#  Adamic-Adar
aa_scores_sorted = sorted(aa_scores, key=lambda x: x[2], reverse=True)
top_pred = aa_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Adamic-Adar):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Adamic-Adar):
Karen Christie - David P. Hill : 28.737172110552162
J. Michael Cherry - Harold Drabkin : 28.615917583946153
Rex L. Chisholm - David P. Hill : 27.94936359048647
Stacia R. Engel - Karen Christie : 27.688365666305607
Penelope Garmiri - Rizwan Ishtiaq : 27.604713310630917
Chris Mungall - David P. Hill : 27.50252874740525
Marc Feuermann - Lucila Aimo : 27.446679288363605
Robert S. Fulton - Asif Chinwalla : 26.78430298822324
M. Eileen Dolan - Rex L. Chisholm : 26.669196604365485
Karen Christie - Judith A. Blake : 26.414047556107864


### Step 5: Xuất file nodes / edges cho Gephi
Nodes: danh sách tác giả + các centrality

Edges: toàn bộ cạnh + weight

In [12]:
# Centrality
deg_c = nx.degree_centrality(G)
bet_c = nx.betweenness_centrality(G)
close_c = nx.closeness_centrality(G)
largest_cc = max(nx.connected_components(G), key=len)
G_sub = G.subgraph(largest_cc)
eig_c_sub = nx.eigenvector_centrality(G_sub, max_iter=5000)
eig_c = {n: eig_c_sub.get(n, 0.0) for n in G.nodes()}

# Nodes
nodes = pd.DataFrame({
    "Id": list(G.nodes()),
    "Degree": [deg_c[n] for n in G.nodes()],
    "Betweenness": [bet_c[n] for n in G.nodes()],
    "Closeness": [close_c[n] for n in G.nodes()],
    "Eigenvector": [eig_c[n] for n in G.nodes()]
})
nodes.to_csv("Project1B_coauthor_nodes_gephi.csv", index=False, encoding="utf-8")

# Edges
edges = pd.DataFrame(list(G.edges(data=True)), columns=["Source", "Target", "Weight"])
edges["Weight"] = edges["Weight"].apply(lambda x: x if isinstance(x, (int,float)) else x.get("Weight",1))
edges.to_csv("Project1B_coauthor_edges_gephi.csv", index=False, encoding="utf-8")


### Step 6: Trực quan trên Gephi

Import nodes → Nodes table

Import edges → Edges table

Partition theo community (nếu tính được)

Size theo Degree / Eigenvector / Betweenness

Layout ForceAtlas2 hoặc Fruchterman-Reingold